In [264]:
# 傷病名分類分析
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

# データ読み込み
df = pd.read_csv('../output/傷病名一覧.csv', encoding='utf-8')
print(f"データ総数: {len(df):,}件")

# float型の値を文字列に変換し、NANを適切に処理
def preprocess_disease_name(name):
    if pd.isna(name):
        return "不明"
    if isinstance(name, (int, float)):
        return str(int(name))
    return str(name)

df['傷病名'] = df['傷病名'].apply(preprocess_disease_name)

# 傷病名の正規化関数
def simplify_disease_name(name):
    
    # 括弧内の詳細情報を除去
    name = re.sub(r'[\(（].*?[\)）]', '', name)
    # 疑いの表現を統一
    name = re.sub(r'(の疑い|疑い|疑|と思われる)', '疑い', name)
    return name.strip()

#疼痛について、運動器の痛みは整形外科にした
#外傷について、硬膜外血腫は神経に入れた
# 分類ルール
rules = {
    '外傷系': r'(咬傷|骨折|打撲|裂傷|挫傷|外傷|脱臼|擦過傷|刺傷|切創|挫創|熱傷|捻挫|脳震盪|心臓震盪|頸髄損傷|頚椎損傷|脊髄損傷|頸椎損傷|頭部血腫|圧挫症候群|腹腔内出血)',
    '整形外科':r'(腰痛|椎間板ヘルニア|頸椎ヘルニア|腰椎ヘルニア|椎症|脊柱管狭窄症|坐骨神経痛|肋間神経痛|関節痛|膝部痛|膝痛|大腿部痛|下肢痛|関節炎|関節症|腱断裂|リウマチ)',
    '循環器系': r'(硬膜外血腫|大動脈解離|大動脈瘤|大動脈瘤破裂|心筋梗塞|虚血性心疾患|急性冠症候群|不整脈|高血圧|狭心症|心不全|心筋症|血圧|房室ブロック|心房細動|心房粗動|心室頻拍|上室性頻拍|洞性頻脈|上室頻拍|心室性期外収縮|肺血栓塞栓症|肺塞栓症|洞不全症候群|徐脈|弁狭窄|弁閉鎖不全|深部静脈血栓症)',
    '脳神経系': r'(脳卒中|脳梗塞|ラクナ梗塞|脳幹梗塞|脳塞栓症|脳幹出血|被殻出血|視床出血|皮質下出血|脳血管障害|脳動脈瘤|脳出血|くも膜下出血|脳腫瘍|水頭症|てんかん|痙攣|意識障害|頭痛|脳虚血|ＴＩＡ|パーキンソン|けいれん|構音障害|呂律障害|痺れ|歩行障害|神経麻痺|筋萎縮性側索硬化症)',
    '消化器系': r'(腹痛|胃炎|胆石|腸炎|下痢|嘔吐|消化管|吐血|下血|血便|急性腹症|腸|イレウス|胆管|胆嚢|胆のう|腹膜炎|憩室炎|虫垂炎|食道静脈瘤|マロリーワイス症候群|憩室出血|消化器出血|逆流性食道炎|胃潰瘍|急性胃粘膜病変|潰瘍性大腸炎|クローン|膵炎|肝硬変|肝機能障害|閉塞性黄疸|肝性脳症|膵臓癌|膵癌|胃癌|肝癌|肝臓癌|食道癌|食道裂孔ヘルニア|鼠径ヘルニア|肝臓癌|肝膿瘍)',
    '呼吸器系': r'(肺炎|気管支炎|喘息|呼吸困難|咳|痰|気道|過換気|肺癌|肺がん|慢性閉塞性肺疾患|気管支拡張症|ＣＯＰＤ|ＣＯ２ナルコーシス|肺気腫|気胸|胸膜炎|膿胸|乳癌)',
    '血液内科': r'(リンパ腫|白血病|多発性骨髄腫|再生不良性貧血|特発性血小板減少性紫斑病|悪性リンパ腫|骨髄異形成症候群|骨髄線維症|骨髄腫|血小板減少|白血球減少|汎血球減少|血友病|溶血性貧血|ヘモグロビン低下|鉄欠乏性貧血|溶血|血栓性血小板減少性紫斑病)',
    '感染症': r'(感染症|敗血症|髄膜炎|発熱|熱発|インフルエンザ|咽頭炎|感冒|悪寒|蜂窩織炎|帯状疱疹|非結核性抗酸菌症|扁桃炎|扁桃周囲膿瘍|急性喉頭蓋炎)',
    '代謝系': r'(糖尿病|電解質|脱水|低血糖|高血糖|痛風|偽痛風|低カリウム血症|低ナトリウム血症|高カリウム血症|高アンモニア血症)',
    '中毒': r'(中毒|過量摂取|アルコール|薬物)',
    '精神系': r'(統合失調|双極性障害|うつ病|不安|精神|認知症|心因反応|解離性障害|パニック障害|パニック発作|せん妄|うつ病|鬱病|不眠症|適応障害)',
    '産婦人科系': r'(分娩|妊娠|流産|性器出血|流産|早産|切迫流産|陣痛|切迫早産|生理痛|月経痛|月経困難症|破水|子宮筋腫|卵巣|子宮)',
    '小児科系': r'(熱性痙攣|百日咳|川崎病|クループ症候群)',
    '泌尿器系': r'(腎臓癌|腎癌|膀胱|尿|前立腺|排尿|水腎|腎盂腎炎|腎結石)',
    '耳鼻科系': r'(鼻出血|メニエール|末梢性眩暈)',
    '老年' : r'(便秘|誤嚥|衰弱|体動困難|歩行困難|起立困難|老衰|褥瘡|廃用症候群)',
    '非特異的愁訴' : r'(喀血|嘔気|悪心|振戦|不随意運動|食欲不振|食思不振|貧血|黄疸|めまい|過呼吸|動悸|呼吸苦|気分不快|胸部不快感|胸部違和感|胸部圧迫感|倦怠感|頭重感|自律神経失調症|ふらつき|全身の痛み|全身の震え|新型コロナワクチン副反応)',
    '心肺停止' : r'(心肺停止|ＣＰＡ|心停止)',
    '腎不全' : r'(腎不全)',
    '痛み' : r'(広背筋痛|下腹部痛|上腹部痛|胃痛|背部痛|心窩部痛|胸部痛|胸痛|後頸部痛|腰部痛|頸部痛|季肋部痛|左側腹部痛|右側腹部痛|胸部絞扼感|後頭部痛)',
    '非外傷性救急' : r'(溺水|呼吸不全|窒息|誤飲|失神|意識消失|肺水腫|胸水貯留|胸水|アナフィラキシー|薬疹|アレルギー|蕁麻疹|ショック|迷走神経反射|脱力|不明熱|低体温|熱中症|低酸素血症|横紋筋融解症|栄養失調|低栄養)',
    '自損' : r'(自殺|自傷|縊頸|過量服薬|多量服薬)',

}

# 分類関数
def classify_disease(name):
    for category, pattern in rules.items():
        if re.search(pattern, name):
            return category
    return 'その他'

# 傷病名の正規化
df['normalized_name'] = df['傷病名'].apply(simplify_disease_name)

# 分類の実行
df['category'] = df['normalized_name'].apply(classify_disease)

# 分類結果の集計
category_counts = df['category'].value_counts()

# 結果の表示
print("\n=== 分類結果 ===")
print("\n各カテゴリの件数:")
for category, count in category_counts.items():
    print(f"{category}: {count:,}件")

# API呼び出し必要数の計算
api_calls_needed = int(category_counts.get('その他', 0))
api_calls_percentage = (api_calls_needed / len(df)) * 100

print(f"\n必要なAPI呼び出し回数: {api_calls_needed:,}回")
print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")
"""
# 可視化
plt.figure(figsize=(12, 6))
sns.barplot(x=category_counts.values, y=category_counts.index)
plt.title('傷病名カテゴリ別の件数')
plt.xlabel('件数')
plt.ylabel('カテゴリ')
plt.tight_layout()
plt.show()
"""

# その他に分類された例の確認
print("\n=== 「その他」に分類された傷病名の例（先頭20件）===")
other_examples = df[df['category'] == 'その他']['傷病名'].value_counts().head(20)
for disease, count in other_examples.items():
    print(f"{disease}: {count:,}件")

データ総数: 172,320件

=== 分類結果 ===

各カテゴリの件数:
外傷系: 38,551件
脳神経系: 20,742件
消化器系: 18,182件
呼吸器系: 13,543件
循環器系: 12,622件
非外傷性救急: 11,647件
感染症: 10,725件
非特異的愁訴: 10,421件
その他: 6,905件
整形外科: 4,577件
泌尿器系: 4,431件
代謝系: 3,796件
痛み: 3,516件
心肺停止: 3,310件
中毒: 2,747件
老年: 1,903件
精神系: 1,603件
耳鼻科系: 1,130件
産婦人科系: 1,056件
腎不全: 403件
血液内科: 264件
小児科系: 129件
自損: 117件

必要なAPI呼び出し回数: 6,905回
全サンプルに対する割合: 4.01%

=== 「その他」に分類された傷病名の例（先頭20件）===
歯肉出血: 22件
右鼠径部痛: 22件
左鼠径部痛: 21件
胸部不快: 20件
腹水: 19件
口腔内出血: 19件
左殿部痛: 18件
右殿部痛: 18件
右肩痛: 18件
殿部痛: 17件
頻脈: 17件
左肩痛: 16件
突発性難聴: 16件
食道異物: 15件
縦隔気腫: 15件
歯痛: 15件
不定愁訴: 15件
肺線維症: 15件
結核: 14件
右下腿部痛: 14件


In [202]:
import numpy as np
np.savetxt("../output/残り病名.csv",df[df['category'] == 'その他']['傷病名'].unique(), fmt="%s")

In [ ]:
# 緊急度・重症度による分類分析
# 無理そう

import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import numpy as np

# データ読み込み
df = pd.read_csv('../output/傷病名一覧.csv', encoding='utf-8')
print(f"データ総数: {len(df):,}件")

# データの前処理
def preprocess_disease_name(name):
    if pd.isna(name):
        return "不明"
    if isinstance(name, (int, float)):
        return str(int(name))
    return str(name)

df['傷病名'] = df['傷病名'].apply(preprocess_disease_name)

# 緊急度・重症度の分類ルール
severity_rules = {
    '最重症': [
        r'(心肺停止|CPA|心停止)',
        r'(ショック|意識レベル|JCS300|JCS200|GCS[1-6])',
        r'(大量出血|多発性外傷|開放性骨折)',
        r'(くも膜下出血|脳出血|急性大動脈解離)',
        r'(重症熱傷|広範囲熱傷|気道熱傷)',
        r'(窒息|気道閉塞|アナフィラキシー)'
    ],
    '重症': [
        r'(心筋梗塞|狭心症|不整脈)',
        r'(脳梗塞|脳卒中|片麻痺)',
        r'(意識障害|JCS100|JCS20|GCS[7-9])',
        r'(呼吸困難|喘息発作|気管支喘息)',
        r'(骨折|脱臼|重度の打撲)',
        r'(急性腹症|消化管出血|吐血|下血)',
        r'(敗血症|髄膜炎|重症感染症)'
    ],
    '中等症': [
        r'(高血圧|糖尿病|腎不全)',
        r'(腹痛|胃腸炎|食中毒)',
        r'(発熱|頭痛|めまい)',
        r'(切傷|擦過傷|捻挫)',
        r'(鼻出血|耳痛|腰痛)',
        r'(嘔吐|下痢|脱水症)'
    ],
    '軽症': [
        r'(風邪|かぜ|感冒)',
        r'(打撲|擦り傷|突き指)',
        r'(皮膚炎|湿疹|かゆみ)',
        r'(軽度の腹痛|軽度の頭痛)',
        r'(不眠|不安|疲労)'
    ]
}

def classify_severity(name):
    for severity, patterns in severity_rules.items():
        if any(re.search(pattern, name, re.IGNORECASE) for pattern in patterns):
            return severity
    return '要判断'

# 分類の実行
df['severity'] = df['傷病名'].apply(classify_severity)

# 重症度の順序を定義
severity_order = ['最重症', '重症', '中等症', '軽症', '要判断']
df['severity'] = pd.Categorical(df['severity'], categories=severity_order, ordered=True)

# 分類結果の集計
severity_counts = df['severity'].value_counts().reindex(severity_order)

# 結果の表示
print("\n=== 分類結果 ===")
print("\n緊急度・重症度別の件数:")
for severity in severity_order:
    count = severity_counts[severity]
    print(f"{severity}: {count:,}件")

# API呼び出し必要数の計算
api_calls_needed = int(severity_counts.get('要判断', 0))
api_calls_percentage = (api_calls_needed / len(df)) * 100

print(f"\n必要なAPI呼び出し回数: {api_calls_needed:,}回")
print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")

# 可視化（順序を考慮）
plt.figure(figsize=(10, 6))
sns.barplot(x=severity_counts.values, y=severity_counts.index, order=severity_order)
plt.title('緊急度・重症度別の件数分布')
plt.xlabel('件数')
plt.ylabel('重症度')
plt.tight_layout()
plt.show()

# システム分類関数
def classify_system(name):
    system_rules = {
        '外傷系': r'(骨折|打撲|裂傷|挫傷|外傷|脱臼|擦過傷|刺傷|切創)',
        '循環器系': r'(心筋梗塞|不整脈|高血圧|狭心症|心不全|心臓|血圧|動悸|胸痛)',
        '脳神経系': r'(脳梗塞|くも膜下出血|てんかん|痙攣|めまい|失神|意識障害|頭痛)',
        '消化器系': r'(腹痛|胃炎|胆石|腸炎|下痢|嘔吐|消化管|吐血|下血)',
        '呼吸器系': r'(肺炎|気管支炎|喘息|呼吸困難|咳|痰|気道)'
    }
    
    for system, pattern in system_rules.items():
        if re.search(pattern, name):
            return system
    return 'その他'

df['system'] = df['傷病名'].apply(classify_system)

# クロス集計表の作成（重症度の順序を考慮）
cross_tab = pd.crosstab(df['system'], df['severity'])
print("\n=== システム分類と重症度のクロス集計 ===")
print(cross_tab[severity_order])  # 列の順序を指定

# 要判断に分類された例の確認
print("\n=== 「要判断」に分類された傷病名の例（先頭20件）===")
undetermined_examples = df[df['severity'] == '要判断']['傷病名'].value_counts().head(20)
for disease, count in undetermined_examples.items():
    print(f"{disease}: {count:,}件")

In [ ]:
import pandas as pd
import re
from anthropic import Anthropic
import asyncio
from typing import List, Dict
import time
from tqdm import tqdm
from pathlib import Path

# Anthropic APIの設定
anthropic = Anthropic(api_key='YOUR_API_KEY')

# 基本的な臓器系統分類ルール
system_rules = {
    '循環器系': r'(心臓|血圧|不整脈|動悸|胸痛|心筋|血管)',
    '呼吸器系': r'(肺|気管|呼吸|喘息|咳|痰)',
    '消化器系': r'(胃|腸|肝臓|胆嚢|膵臓|腹痛|嘔吐)',
    '脳神経系': r'(脳|神経|意識|めまい|頭痛|痙攣)',
    '筋骨格系': r'(骨|関節|筋肉|腰痛|背部痛)',
    '泌尿器系': r'(腎臓|膀胱|尿)',
    '内分泌系': r'(甲状腺|副腎|糖尿病|ホルモン)',
    '血液系': r'(貧血|出血|白血球|血小板)',
    '皮膚系': r'(皮膚|発疹|かゆみ|湿疹)'
}

class PromptTemplate:
    def __init__(self, template_path: str):
        """プロンプトテンプレートの初期化"""
        self.template = self._load_template(template_path)
    
    def _load_template(self, template_path: str) -> str:
        """テンプレートファイルの読み込み"""
        try:
            with open(template_path, 'r', encoding='utf-8') as f:
                return f.read()
        except FileNotFoundError:
            raise FileNotFoundError(f"プロンプトテンプレートファイルが見つかりません: {template_path}")
    
    def format(self, diseases: List[str]) -> str:
        """プロンプトの生成"""
        diseases_list = "\n".join([f"{i+1}. {d}" for i, d in enumerate(diseases)])
        return self.template.format(
            count=len(diseases),
            diseases_list=diseases_list
        )

def classify_by_rules(name: str) -> str:
    """ルールベースでの分類"""
    for system, pattern in system_rules.items():
        if re.search(pattern, name):
            return system
    return '要API判定'

async def process_batch(batch: List[str], prompt_template: PromptTemplate) -> List[str]:
    """バッチ処理でHaiku APIを呼び出す"""
    try:
        prompt = prompt_template.format(batch)
        response = await anthropic.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        results = parse_haiku_response(response.content, len(batch))
        return results
    except Exception as e:
        print(f"Error processing batch: {e}")
        return ['エラー'] * len(batch)

def parse_haiku_response(response: str, expected_length: int) -> List[str]:
    """Haiku APIのレスポンスをパース"""
    results = ['その他'] * expected_length  # デフォルト値を設定
    try:
        lines = response.strip().split('\n')
        for line in lines:
            if '.' in line:
                parts = line.strip().split('.')
                if len(parts) >= 2:
                    idx = int(parts[0]) - 1
                    system = parts[1].strip()
                    if 0 <= idx < expected_length:
                        results[idx] = system
    except Exception as e:
        print(f"Error parsing response: {e}")
    return results

async def process_all_data(df: pd.DataFrame, prompt_template: PromptTemplate, batch_size: int = 20):
    """全データの処理"""
    # まずルールベースで分類
    df['organ_system'] = df['傷病名'].apply(classify_by_rules)
    
    # API判定が必要なケースを抽出
    need_api = df[df['organ_system'] == '要API判定']
    total_api_cases = len(need_api)
    print(f"API判定が必要なケース: {total_api_cases:,}件")
    
    # バッチ処理の準備
    need_api_indices = need_api.index
    results = []
    batches = [need_api['傷病名'].iloc[i:i+batch_size].tolist() 
               for i in range(0, len(need_api), batch_size)]
    
    # プログレスバーでバッチ処理の進捗を表示
    api_calls = 0
    for batch in tqdm(batches, desc="API処理中"):
        batch_results = await process_batch(batch, prompt_template)
        results.extend(batch_results)
        api_calls += 1
        await asyncio.sleep(0.5)  # レート制限対策
    
    # 結果をデータフレームに反映
    df.loc[need_api_indices, 'organ_system'] = results
    
    return df, api_calls

async def main():
    # プロンプトテンプレートの読み込み
    prompt_template = PromptTemplate('prompt_template.txt')
    
    # データ読み込み
    df = pd.read_csv('傷病名一覧.csv', encoding='utf-8')
    print(f"データ総数: {len(df):,}件")
    
    # 処理開始時間
    start_time = time.time()
    
    # データ処理
    processed_df, api_calls = await process_all_data(df, prompt_template)
    
    # 処理時間の計算
    process_time = time.time() - start_time
    
    # 結果の集計と表示
    system_counts = processed_df['organ_system'].value_counts()
    print("\n=== 分類結果 ===")
    for system, count in system_counts.items():
        print(f"{system}: {count:,}件")
    
    print(f"\nAPI呼び出し回数: {api_calls:,}回")
    print(f"処理時間: {process_time:.2f}秒")
    print(f"バッチ処理による削減率: {((1 - api_calls / len(df)) * 100):.1f}%")
    
    # 結果の保存
    processed_df.to_csv('classified_diseases_haiku.csv', index=False)

if __name__ == "__main__":
    asyncio.run(main())